In [1]:
import pandas as pd
import numpy as np
import chemivec
from chem_render import indigo

import dask.dataframe as dd

%load_ext autoreload
%autoreload 2

df = pd.read_pickle(f"./ORD/uspto_rxn.pkl.zst")
mini = df['rxn_smiles'].values[:10000]

In [7]:
#np.save("test_10000.npy", df.rxn_smiles.values[:10000])

In [16]:
SUZUKI_PATTERN = "[B;X3,4]-[C,c:1].[C,c:2]-[Cl,Br,I,$([O]-S)]>>[C,c:1]-[C,c:2]"

%timeit chemivec._rxn_match(mini, SUZUKI_PATTERN, "DAYLIGHT-AAM")

6.64 s ± 44.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%time
# Indigo search using atom mapping in target reaction.

query = indigo.loadReactionSmarts(SUZUKI_PATTERN)
query.optimize()

def rxn_match(rxn_smiles) -> bool:
    return True if indigo.substructureMatcher(indigo.loadReaction(rxn_smiles), "DAYLIGHT-AAM").match(query) else False

dds = dd.from_pandas(df['rxn_smiles'], npartitions=240)
suzuki_indigo_aam = dds.apply(rxn_match,
                              # args=[query,),
                              # rxn_query=query,
                              meta=pd.Series(dtype=bool)).compute(scheduler='threads')
suzuki_indigo_aam.sum()

CPU times: total: 0 ns
Wall time: 0 ns
CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
%%time
suzuki_vec = chemivec._rxn_match(df['rxn_smiles'].values, SUZUKI_PATTERN, "DAYLIGHT-AAM")
suzuki_vec.sum()

CPU times: total: 0 ns
Wall time: 0 ns
CPU times: total: 0 ns
Wall time: 0 ns
